In [53]:
import pandas as pd
import numpy as np

#load excel file
file_path = "RAW_data.xlsx"
df = pd.read_excel(file_path)

In [54]:
#show top 5 rows
df.head()


,Nitrogen - total (mg/L),Phosphorus - total (mg/L),Solids - total suspended @ 105 C (mg/L),Turbidity,Oxygen - dissolved saturation (%saturation),Oxygen - dissolved (mg/L),pH,Temperature - Water (°C),Conductivity
0,0.38,0.033,17.0,21.7,90.1,9.06,7.8,14.0,440.0
1,0.61,0.046,21.0,16.9,121.0,11.76,8.5,15.8,500.0
2,0.90,0.190,190.0,295.0,85.0,8.11,7.3,16.8,180.0
3,0.40,0.043,18.0,28.1,97.3,8.53,8.1,21.8,390.0
4,0.40,0.047,21.0,29.7,97.6,8.04,8.4,25.1,480.0


In [55]:
#show data summary
df.describe()

,Nitrogen - total (mg/L),Phosphorus - total (mg/L),Solids - total suspended @ 105 C (mg/L),Turbidity,Oxygen - dissolved saturation (%saturation),Oxygen - dissolved (mg/L),pH,Temperature - Water (°C),Conductivity
count,1018.000000,1027.000000,1042.000000,1019.000000,1024.000000,1022.000000,1019.000000,1025.000000,1026.000000
mean,0.967066,0.149869,63.929942,79.442875,86.088086,8.030607,7.904249,19.210827,462.406140
std,0.858202,0.271293,167.643909,155.914789,23.025060,2.239946,0.426838,6.303131,313.077459
min,0.050000,0.010000,5.000000,0.800000,5.100000,0.490000,6.310000,5.608000,73.500000
25%,0.500000,0.040000,11.000000,9.390000,74.075000,6.645000,7.630000,13.454000,252.425000
50%,0.800000,0.080000,26.000000,23.100000,87.500000,8.205000,7.900000,19.482000,379.850000
75%,1.100000,0.160000,61.750000,74.300000,96.775000,9.650000,8.170000,24.000000,555.750000
max,9.000000,4.100000,2840.000000,2400.000000,200.000000,17.080000,10.160000,38.680000,2061.000000


In [56]:
#rename
df[['Conductivity']] = df [['Conductivity ']]


In [57]:
#taking only columns of interest
df_sliced = df[['Conductivity', 'pH', 'Turbidity']]
df_sliced.head() # display top 5 rows

,Conductivity,pH,Turbidity
0,440.0,7.8,21.7
1,500.0,8.5,16.9
2,180.0,7.3,295.0
3,390.0,8.1,28.1
4,480.0,8.4,29.7


In [58]:
#check for attributes with null values
df_sliced.isnull().sum()


Conductivity    22
pH              29
Turbidity       29
dtype: int64

In [59]:
#remove null values 
df_cleaned = df_sliced.dropna()
df_cleaned.head()

,Conductivity,pH,Turbidity
0,440.0,7.8,21.7
1,500.0,8.5,16.9
2,180.0,7.3,295.0
3,390.0,8.1,28.1
4,480.0,8.4,29.7


In [60]:
#compare sizes before and after cleaning
print("Original size:", df_sliced.shape)
print("Cleaned size:", df_cleaned.shape)

Original size: (1048, 3)
Cleaned size: (1012, 3)


In [61]:
#save cleaned data to csv file
df_cleaned.to_csv("cleaned_water_quality_data.csv", index=False)